## Imports

In [0]:
import numpy
import pandas 
import keras
import tensorflow.contrib.eager as tfe
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle



## Fixing Random Seed

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


## Loading Data

In [0]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [0]:
X.shape[1]

## Label Encoding

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
reverse_encoding = encoder.inverse_transform(encoded_Y) # reversing the encoding for fun

In [0]:
encoded_Y

# Baseline Model (Sequential)

In [0]:
# baseline model
def create_baseline():
  # create model, write code below
  model = models.Sequential()
  model.add(layers.Dense(60, activation ='relu', input_shape = (X.shape[1],))) # number of features (60) are used as vector in input shape
  model.add(layers.Dense(1, activation ='sigmoid'))
	
  # Compile model, write code below
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
	
  return model
 


## Evaluatuon 

In [0]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Results: 83.71% (6.13%)


In [0]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 85.59% (7.46%)


# Smaller Model (Sequential)

In [0]:
# smaller model
def create_smaller():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(30, activation ='relu', input_shape = (X.shape[1],))) # number of features (60) are used as vector in input shape
  model.add(layers.Dense(1, activation ='sigmoid'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Smaller: 85.06% (7.61%)


# Larger Model (Sequential)

In [0]:
# larger model
def create_larger():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(60, activation ='relu', input_shape = (X.shape[1],))) # number of features (60) are used as vector in input shape
  model.add(layers.Dense(30, activation ='relu'))
  model.add(layers.Dense(1, activation ='sigmoid'))
	# Compile model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 83.59% (4.65%)


# Fully Scaled (Sequential)

In [0]:
# fully scaled up model
def fully_scaled():
  # create model
  model = models.Sequential()
  model.add(layers.Dense(512, activation ='relu', input_shape = (X.shape[1],))) # number of features (60) are used as vector in input shape
  model.add(layers.Dense(256, activation ='relu'))
  model.add(layers.Dense(128, activation ='relu'))
  model.add(layers.Dense(128, activation ='relu'))
  model.add(layers.Dense(64, activation ='relu'))
  model.add(layers.Dense(32, activation ='relu'))
  model.add(layers.Dense(16, activation ='relu'))
  model.add(layers.Dense(1, activation ='sigmoid'))
  # Compile model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=fully_scaled, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Fully Scaled: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



Fully Scaled: 87.52% (6.57%)


# Tuned Model (Sequential)

---



## Trial Combination 

### Tuned: 84.52% (5.93%) l1 =90, l2= 60, output= 1

### Tuned: 85.04% (6.41%) 100 100 1

### Tuned: 85.06% (7.03%) 90 35 1

### Tuned: 85.13% (7.79%) 80 30 1

### Tuned: 85.54% (6.49%) 100 30 1

### Tuned: 85.56% (8.58%) 60 32 16 8 1

### Tuned: 85.97% (5.65%) 90 40 1

### Tuned: 86.04% (9.79%) 512 128 64 16 1 

### Tuned: 86.54% (6.74%) 90 20 1

### *Tuned: 86.99% (5.43%) 90 30 1*

---



### Loss_Func = binary_crossentropy, Optimizer = Adam


In [0]:
def tuned_model_1():
  # create model, write code below
  model = models.Sequential()
  model.add(layers.Dense(90, activation ='relu', input_shape = (X.shape[1],)))
  model.add(layers.Dense(30, activation ='relu'))
  model.add(layers.Dense(1, activation ='sigmoid'))
	
  # Compile model, write code below
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tuned_model_1, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Tuned_1: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tuned_1: 86.99% (5.43%)


# Functional API

In [0]:
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras import layers
from keras import Input

In [0]:
def create_baseline_functional():
  input_tensor = Input(shape=(60,))
  x = layers.Dense(60, activation='relu')(input_tensor)
#x = layers.Dense(1, activation='relu')(x)
  output_tensor = layers.Dense(1, activation='sigmoid')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  
  return model

# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline_functional, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.59% (7.46%)


In [0]:
def create_smaller_functional():
  input_tensor = Input(shape=(60,))
  x = layers.Dense(30, activation='relu')(input_tensor)
  output_tensor = layers.Dense(1, activation='sigmoid')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  
  return model

# evaluate smaller model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller_functional, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.06% (7.61%)


In [0]:
# larger model
def create_larger_functional():
# create model
  input_tensor = Input(shape=(60,))
  x = layers.Dense(60, activation='relu')(input_tensor)
  x = layers.Dense(30, activation='relu')(x)
  output_tensor = layers.Dense(1, activation='sigmoid')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
# Compile model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model
# evaluate larger model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger_functional, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 83.59% (4.65%)


In [0]:
def create_fully_scaled_functional():
  input_tensor = Input(shape=(60,))
  x = layers.Dense(512, activation='relu')(input_tensor)
  x = layers.Dense(256, activation='relu')(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(128, activation='relu')(x)
  x = layers.Dense(64, activation='relu')(x)
  x = layers.Dense(32, activation='relu')(x)
  x = layers.Dense(16, activation='relu')(x)
  output_tensor = layers.Dense(1, activation='sigmoid')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  
  return model
# evaluate fully scaled model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_fully_scaled_functional, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Fully Scaled: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Fully Scaled: 84.09% (8.11%)


In [0]:
def create_tuned_model_functional():
  # create model, write code below
  input_tensor = Input(shape=(60,))
  x = layers.Dense(90, activation='relu')(input_tensor)
  x = layers.Dense(30, activation='relu')(x)
  output_tensor = layers.Dense(1, activation='sigmoid')(x)
# The Model class turns an input tensor and output tensor into a model
  model = Model(input_tensor, output_tensor)
  
  # Compile the model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  
  return model
# evaluate tuned model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_tuned_model_functional, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Tuned: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Tuned: 86.49% (7.11%)


# Model Subclassing

In [0]:
class MyBaselineModel(keras.Model):
  
  def __init__(self):
    super(MyBaselineModel,self).__init__('my_baseline')
  #  self.units = 60
    self.dense1 = keras.layers.Dense(60, activation='relu')
    self.dense2 = keras.layers.Dense(1, activation='sigmoid')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    
def create_baseline_subclass_model():
  inputs = keras.Input(shape=(60,))
  mymodel = MyBaselineModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.59% (7.46%)


In [0]:
class MySmallerModel(keras.Model):
  
  def __init__(self):
    super(MySmallerModel,self).__init__('my_Smaller')
    self.dense1 = keras.layers.Dense(30, activation='relu')
    self.dense2 = keras.layers.Dense(1, activation='sigmoid')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)
    
def create_smaller_subclass_model():
  inputs = keras.Input(shape=(60,))
  mymodel = MySmallerModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

# evaluate smaller model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 83.09% (6.41%)


In [0]:
class MylargerModel(keras.Model):
  
  def __init__(self):
    super(MylargerModel,self).__init__('my_larger')
    self.dense1 = keras.layers.Dense(60, activation='relu')
    self.dense2 = keras.layers.Dense(30, activation='relu')
    self.dense3 = keras.layers.Dense(1, activation='sigmoid')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)
    
def create_larger_subclass_model():
  inputs = keras.Input(shape=(60,))
  mymodel = MylargerModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

# evaluate larger model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 85.09% (7.25%)


In [0]:
class MyFullyScaledModel(keras.Model):
  
  def __init__(self):
    super(MyFullyScaledModel,self).__init__('my_FullyScaled')
    self.dense1 = keras.layers.Dense(512, activation='relu')
    self.dense2 = keras.layers.Dense(256, activation='relu')
    self.dense3 = keras.layers.Dense(128, activation='relu')
    self.dense4 = keras.layers.Dense(128, activation='relu')
    self.dense5 = keras.layers.Dense(64, activation='relu')
    self.dense6 = keras.layers.Dense(32, activation='relu')
    self.dense7 = keras.layers.Dense(16, activation='relu')
    self.dense8 = keras.layers.Dense(1, activation='sigmoid')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.dense5(x)
    x = self.dense6(x)
    x = self.dense7(x)
    return self.dense8(x)
    
def create_Fully_Scaled_subclass_model():
  inputs = keras.Input(shape=(60,))
  mymodel = MyFullyScaledModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

# evaluate fully scaled model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_Fully_Scaled_subclass_model, epochs=300, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Fully Scaled: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Fully Scaled: 86.04% (6.28%)


In [0]:
class MyTunedModel(keras.Model):
  
  def __init__(self):
    super(MyTunedModel,self).__init__('my_TunedModel')
    self.dense1 = keras.layers.Dense(90, activation='relu')
    self.dense2 = keras.layers.Dense(30, activation='relu')
    self.dense3 = keras.layers.Dense(1, activation='sigmoid')
    
  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)
    
def create_Tuned_subclass_model():
  inputs = keras.Input(shape=(60,))
  mymodel = MyTunedModel()
  outputs = mymodel.call(inputs)
    
  model = keras.Model(inputs,outputs)
    
# compiling model
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

# evaluate tuned model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_Tuned_subclass_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Tuned: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Tuned: 86.90% (9.89%)


# Without Scikit Learn Library

In [0]:
def tuned_model():
  # create model, write code below
  model = models.Sequential()
  model.add(layers.Dense(90, activation ='relu', input_shape = (X.shape[1],)))
  model.add(layers.Dense(30, activation ='relu'))
  model.add(layers.Dense(1, activation ='sigmoid'))
	
  # Compile model, write code below
  model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [0]:
# shuffling data
dataframe = shuffle(dataframe)

# checking data
dataset = dataframe.values

# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
reverse_encoding = encoder.inverse_transform(encoded_Y) # reversing the encoding for fun



In [0]:
k = 10
num_val_samples = len(X) // 10
num_epochs = 100
all_scores = numpy.array([])

for i in range(k):
  print('processing fold # ', i+1)
  # prepare the validation data: data from partition # k
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
  
  # prepare the training data: data from data - k
  partial_train_data = numpy.concatenate(                    
      [X[:i * num_val_samples],
      X[(i + 1 ) * num_val_samples:]],
  axis = 0)
  partial_train_targets = numpy.concatenate(
      [encoded_Y[:i * num_val_samples],
      encoded_Y[(i + 1 ) * num_val_samples:]],
  axis = 0)
  
  # Build the Keras Models (already commpiled)
  model = tuned_model()
  
  # Train the model (in silence mode, verbose = 0)
  history = model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 5, verbose = 0)
  
  # Evaluate the model on the validation data
  val_binary_crossentropy, val_accuracy = model.evaluate(val_data, val_targets, verbose = 0)
  
  all_scores = numpy.append(all_scores, val_accuracy)
  print("K-Fold Witout Scikit-learn : (Accuracy)Mean %.2f%% (Error)Std (%.2f%%)" % (all_scores.mean()*100, all_scores.std()*100))
       



processing fold #  1
K-Fold Witout Scikit-learn : (Accuracy)Mean 90.00% (Error)Std (0.00%)
processing fold #  2
K-Fold Witout Scikit-learn : (Accuracy)Mean 90.00% (Error)Std (0.00%)
processing fold #  3
K-Fold Witout Scikit-learn : (Accuracy)Mean 88.33% (Error)Std (2.36%)
processing fold #  4
K-Fold Witout Scikit-learn : (Accuracy)Mean 90.00% (Error)Std (3.54%)
processing fold #  5
K-Fold Witout Scikit-learn : (Accuracy)Mean 89.00% (Error)Std (3.74%)
processing fold #  6
K-Fold Witout Scikit-learn : (Accuracy)Mean 87.50% (Error)Std (4.79%)
processing fold #  7
K-Fold Witout Scikit-learn : (Accuracy)Mean 85.00% (Error)Std (7.56%)
processing fold #  8
K-Fold Witout Scikit-learn : (Accuracy)Mean 85.00% (Error)Std (7.07%)
processing fold #  9
K-Fold Witout Scikit-learn : (Accuracy)Mean 85.56% (Error)Std (6.85%)
processing fold #  10
K-Fold Witout Scikit-learn : (Accuracy)Mean 85.50% (Error)Std (6.50%)
